In [46]:
import fire
import os
import requests
import pandas as pd
from dotenv import load_dotenv
load_dotenv()
import numpy as np
import alpaca_trade_api as alpaca
import questionary
import hvplot.pandas

from qualifier.utils.alpaca_api import alpaca_func

from qualifier.utils.analysis import largest_25_OI
from qualifier.utils.analysis import options_call_bar
from qualifier.utils.analysis import options_put_bar
from qualifier.utils.analysis import put_stats
from qualifier.utils.analysis import call_stats
from qualifier.utils.analysis import ticker_report

from Modules.CleanData import get_data
from qualifier.utils.SQL import add_new_table

# Alpaca API

In [47]:
# Gets Alpaca key and secret from .env.
alpaca_key = os.getenv("ALPACA_API_KEY")
alpaca_secret = os.getenv("ALPACA_SECRET_KEY")

# Specify dates for Alpaca API.
start_date = "2022-09-16"
end_date = start_date

# Specify ticker of stock to get info from Alpaca API.
stock = "AAPL"

In [48]:
# Calling alpaca_func
alpaca_func(alpaca_key, alpaca_secret, start_date, end_date, stock)

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2022-09-16 04:00:00+00:00,151.33,151.35,148.37,150.7,163574426,850046,150.227405


# Tao's Code

In [49]:
# Specify date for Tao's code.
# Tao's code takes year at the end.

date = "09-16-2022"

In [50]:
# Calling get_data()
df = get_data(date)


In [51]:
# Calling largest_25_OI
largest_25_OI(df)

:Bars   [Symbol,Type]   (Open Int)

In [52]:
# Calling options_call_bar
options_call_bar(df)

:Bars   [Symbol,Strike]   (Open Int)

In [53]:
# Calling options_put_bar
options_put_bar(df)

:Bars   [Symbol,Strike]   (Open Int)

In [54]:
options_call_bar(df) * options_put_bar(df).opts(
    title= 'Call to Put Comparison'
)

:Overlay
   .Bars.Call :Bars   [Symbol,Strike]   (Open Int)
   .Bars.Puts :Bars   [Symbol,Strike]   (Open Int)

In [55]:
options_put_bar(df) * options_call_bar(df).opts(
    title= 'Put to Call Comparison'
)

:Overlay
   .Bars.Puts :Bars   [Symbol,Strike]   (Open Int)
   .Bars.Call :Bars   [Symbol,Strike]   (Open Int)

# Single Stock Report from DataFrame.

In [71]:
# Specify dates for Alpaca API.
start_date = "2022-09-12"
end_date = "2022-09-16"

# Specifies ticker from df above. Can single out any stock you like.
ticker = 'TSLA'
stock = ticker

In [72]:
# Runs ticker report on single stock from data.
report = ticker_report(ticker,df)

# Creates ticker df from function
ticker_df = alpaca_func(alpaca_key, alpaca_secret, start_date, end_date, stock)

# Creates plot from df
plot = ticker_df['close'].hvplot.line(x='Time', y='close')

# Creates side by side
# report + plot

In [73]:
report

:Bars   [Strike,Type]   (Open Int)

In [74]:
plot

:Curve   [Time]   (close)

# Stats

In [60]:
call_stats(df)

,Open Int
count,97.000000
mean,16631.793814
std,20918.616662
min,1140.000000
25%,4764.000000
50%,10151.000000
75%,21497.000000
max,146857.000000


In [61]:
put_stats(df)

,Open Int
count,97.000000
mean,16631.793814
std,20918.616662
min,1140.000000
25%,4764.000000
50%,10151.000000
75%,21497.000000
max,146857.000000


# Testing

In [62]:
calls = df[df['Type'] == 'Call'].drop(columns='Price')
calls.head()

,Symbol,Type,Strike,Exp Date,DTE,Bid,Midpoint,Ask,Last,Volume,Open Int,OI Chg,IV,Time,Total Open Int
1427,AAL,Call,13.0,2022-09-16,0.0,0.71,0.76,0.80,0.77,937.0,8574.0,-120,76.72%,2022-09-16,79346.0
1195,AAL,Call,13.5,2022-09-16,0.0,0.21,0.28,0.34,0.26,1141.0,3984.0,-379,27.55%,2022-09-16,79346.0
1433,AAPL,Call,125.0,2022-09-16,0.0,24.80,25.83,26.85,25.75,968.0,21065.0,-117,158.24%,2022-09-16,338171.0
1365,AAPL,Call,130.0,2022-09-16,0.0,20.05,20.35,20.65,20.60,754.0,13320.0,-158,0.00%,2022-09-16,338171.0
1448,AAPL,Call,135.0,2022-09-16,0.0,14.80,15.75,16.70,15.50,566.0,7634.0,-108,193.51%,2022-09-16,338171.0


In [63]:
puts = df[df['Type'] == 'Put'].drop(columns='Price')
puts.head()

,Symbol,Type,Strike,Exp Date,DTE,Bid,Midpoint,Ask,Last,Volume,Open Int,OI Chg,IV,Time,Total Open Int
330,AAL,Put,14.0,2022-09-16,0.0,0.20,0.25,0.30,0.23,6970.0,16286.0,"+1,041",49.33%,2022-09-16,79346.0
309,AAL,Put,14.5,2022-09-16,0.0,0.71,0.76,0.80,0.73,873.0,2082.0,"+1,103",95.30%,2022-09-16,79346.0
1144,AAL,Put,15.0,2022-09-16,0.0,1.21,1.27,1.32,1.21,1374.0,48420.0,-556,147.68%,2022-09-16,79346.0
1016,AAPL,Put,152.5,2022-09-16,0.0,1.65,2.33,3.00,1.78,23955.0,31120.0,"-4,510",62.85%,2022-09-16,338171.0
1001,AAPL,Put,155.0,2022-09-16,0.0,4.15,4.80,5.45,4.35,14856.0,34995.0,"-23,986",32.94%,2022-09-16,338171.0
